- now that you know what a particle filter is and how it works, in this lesson you'll learn some important implementation details
- afterwards, you'll be able to complete a project which involves making a particle filter to localize a self-driving car
- code can be a bit different from theory and math, so I'll go over a few things you need to know, such as how to assign measurements to particular landmarks in a given map

# Pseudocode

<img src="resources/particle_filter_algorithm_pseudocode.png"/>

- line 1 is the initialization step for the filter
  - at the initialization step we estimate our position from GPS input
- line 3 this is the prediction step of the filter, which is analogous to the prediction step of the base filter you saw before
  -  during the prediction step we add the control input (yaw rate & velocity) for all particles
- lines 4 and 5 is the update step, where you will update each particle's weight
  - during the update step, we update our particle weights using map landmark positions and feature measurements
- the second for loop (lines 7 - 10) is the all-important resampling step that you learned about in the previous lesson
  - during resampling we will resample M times (M is range of 0 to length_of_particleArray) drawing a particle i (i is the particle index) proportional to its weight
- once the resampling is done, the new set of particles represents the Bayes filter posterior (line 11) probability
  - we now have a refined estimate of the vehicles position based on input evidence


- the flowchart below represents the steps of the particle filter algorithm as well as its inputs

<img src="resources/particle_filter_algorithm_flowchart.png"/>

# Initialization

- the very first thing you'll need to do to implement a particle filter is initialize all your particles
- you'll have to decide how many particles you want to use
- this parameter is mostly decided empirically, but theoretically, the particle filter will exactly represent the Bayesian posterior distribution as the number of particles approaches infinity
- if you have too few particles, you will not have enough to cover all of the high likelihood positions, so you might miss the correct position
- however, having too many particles will slow down your filter, and prevent it from localizing a self-driving car in real time


- there are two main ways you can initialize your particles
- the first is to sample uniformly across the state space
  - for instance, you could divide the state space into a grid, and put one particle in each cell
  - this approach is not very practical if the state space is too large, such as in the case of a self-driving car, where the state space spans essentially the entire land surface of the earth
- the second approach for initialization is to sample around some kind of initial estimate
  - for a self-driving car, this initial estimate can come from a global positioning system, or GPS
  - if you remember from the very beginning of the localization module, you cannot rely on GPS alone for localization, because of its low accuracy and reduced availability in certain environments
  - however, GPS can be very useful to provide initial rough estimate of your location


- for implementation in the particle filter project, I want you to initialize your particle filter by sampling from a Gaussian distribution, taking into account Gaussian sensor noise around the initial GPS position estimate, and the initial heading estimate

- project implementation
  - particles shall be implemented by sampling a Gaussian distribution, taking into account Gaussian sensor noise around the initial GPS position and heading estimates
  - use the [C++ standard library normal distribution](http://en.cppreference.com/w/cpp/numeric/random/normal_distribution) and [C++ standard library random engine](http://www.cplusplus.com/reference/random/default_random_engine) functions to sample positions around GPS measurements

# Program Gaussian Sampling

- I have provided you with a function that takes a GPS position and initial heading as input
- I want you to print out to the terminal 3 samples from a normal distribution with mean equal to the GPS position and initial heading measurements and standard deviation of $2$ m for the x and y position and $0.05$ radians for the heading of the car

- code is available in `code/01_program_gaussian_sampling/`

# Prediction Step

- the next part of the filter you will need to implement is the prediction step
- for the prediction step, you will use what you learned in the motion models lesson to predict where the car will be at the next time step
- for each particle, you will have to update the particle's location based on velocity yaw rate measurements
- to account for the uncertainty in the control input, in the project you will also add Gaussian noise to the velocity and yaw rate

- I want you to practice calculating the prediction of a car's position, assuming a basic motion model and disregarding sensor  uncertainty for now


- **Q:** Given the car’s last position was at $(102 m, 65 m)$ with a heading of $\dfrac{5\pi}{8}$ radians, the car’s velocity was $110 m/s$, and the car’s yaw rate was $\dfrac{\pi}{8} rad/s$ over the last $0.1$ seconds, what is the car’s new position and heading? Recall the equations for updating x, y and the yaw angle when the yaw rate is not equal to zero.
- **A::**
  - the equations for updating x, y and the yaw angle when the yaw rate is not equal to zero:
    - $x_f = x_0 + \dfrac{v}{ \dot{\theta}}[sin(\theta_0 + \dot{\theta}(dt)) - sin(\theta_0)]$
    - $y_f = y_0 + \dfrac{v}{ \dot{\theta}}[cos(\theta_0) - cos(\theta_0 + \dot{\theta}(dt))]$
    - $\theta_f = \theta_0 + \dot{\theta}(dt)$
  - plugging in all the numbers should give:
    - $102+\frac{110}{\frac{\pi}{8}}[sin(\frac{5\pi}{8}+\frac{\pi}{8}(0.1))-sin(\frac{5\pi}{8})] = 97.59$
    - $65+\frac{110}{\frac{\pi}{8}}[cos(\frac{5\pi}{8})-cos(\frac{5\pi}{8}+\frac{\pi}{8}(0.1))] = 75.08$
    - $\frac{5\pi}{8}+ \frac{\pi}{8}(0.1)= \frac{51\pi}{80}$

# Data Association: Nearest Neighbor

- before we can use landmark measurements of the objects around us to update the belief of our position, we'll have to solve the data association problem
- data association is the problem of matching landmark measurements to objects in the real world, like map landmarks


- let's look at an example to show why data association is so important
- we can assume that we're confident that we have two lidar sensor measurements--which of these two measurements corresponds to the landmark?
- remember that for localization, when sensor measurements--or what we actually see--matches to map landmarks--what we expect to see--this must be the true position of the car


- so how do we pick the right measurement?
- for the self-driving car, you might have a map with landmarks, shown in blue, and lidar measurements, shown in orange
- oftentimes, you have some map landmarks that have multiple lidar measurements that could correspond to it
- to pick the right correspondents, we can use a very simple technique called nearest neighbor
- in this method, simply take the closest measurement as the correct correspondents, like the circled map landmark and lidar measurement

<img src="resources/nearest_neighbor.png"/>

- **Q:** Which of the following characteristics of the input data would help nearest neighbor data association become more effective? 
- **A:** High signal-to-noise ratio for sensors, A very accurate motion model.

# Nearest Neighbor Advantages and Disadvantages

- nearest neighbor is great, because it's easy to understand and implement--just take the closest measurement
- however, simple techniques have some disadvantages in that they are not as robust to difficult scenarios


- for instance, a high density of measurements would make this technique very prone to errors, because one map landmark could be really close to multiple measurements
  - it may be unclear which feature corresponds to the map landmarks

<img src="resources/nearest_neighbor_high_density.png"/>

- in addition, the nearest neighbor algorithm is quite inefficient, because for each landmark you have to search through all of the measurements to find the one with the minimum distance
  - therefore, the complexity is $O(mn)$, where $m$ is the number of map landmarks and $n$ is the number of sensors measurements
  - if there are a lot of sensor measurements, or landmarks, nearest neighbor could become quite slow

<img src="resources/nearest_neighbor_complexity.png"/>

- additionally, if a sensor is very noisy, you could easily match a spurious feature, one that doesn't actually correspond to any object in the real world to a map landmark
- the landmark on the right has an incorrect association due to sensor noise--you can tell that this other match is correct, because it matches the geometry of this other landmark match

<img src="resources/nearest_neighbor_noisy_sensor.png"/>

- another disadvantage of nearest neighbor is that it is not as robust to errors in the vehicle position estimate
- if the vehicle position has some errors, nearest neighbor will also incur some errors
- here, the car is shifted to the right so the measurements are also shifted to the right, as shown with the purple lines
  - however, nearest neighbor will not see this pattern, and will pick some incorrect correspondences, which are circled in black

<img src="resources/nearest_neighbor_position_estimate.png"/>

- another disadvantage of nearest neighbor is that it does not take uncertainties of the sensor into account
- for example, let's say you have a radar sensor that has very precise bearing measurements and not as precise range measurements
- the green ovals here represent the uncertainties in the range and bearing dimensions for these radar measurements
- nearest neighbor would pick this match, but this other match is more likely  given that the sensor is very accurate in the bearing dimension and not as accurate in the range dimension

<img src="resources/nearest_neighbor_uncertanties.png"/>

- let's summarize
  - pros
    - the nearest neighbor data association technique is easy to understand and implement, and it works well in many situations
  - cons
    - it is not as robust to large densities of measurements, sensor noise, or errors in position estimates
    - it can be inefficient to compute, and also does not take sensor uncertainties into account

# Update Step

- you have learned how to incorporate velocity and yaw rate measurements into the particle filter
- but what about sensor measurements, such as readings from radar or lidar?
  - these landmark measurements will be used to compute the update step, which you saw before in the Bayesian filter


- instead of the feature measurements directly affecting the prediction of the state of the car, the measurements will instead inform the weight of each particle
  - the size of each particle correlates with its weight
- one way to update the weights of the particles is to use the multivariate Gaussian probability density function for each measurement and combine the likelihoods of all the measurements by taking their product
  - this function tells us how likely a set of landmark measurements is, given our predicted state of the car and the assumption that the sensors have Gaussian noise
  - we also assume each landmark measurement is independent, so we will take the product of the likelihoods over all measurements
  


- $x_i$ represents the $i^{th}$ landmark measurement for one particular particle
- $\mu_i$ represents the predicted measurement for the map landmark corresponding to the $i^{th}$ measurement
- $m$ is the total number of measurements for one particle
- $\Sigma$ (sigma) is the covariance of the measurement
  - the covariance matrix $\Sigma$ is a symmetric square matrix that contains the variance, or uncertainty, of each variable in the sensor measurement, as well as the covariance, or correlation, between these variables

<img src="resources/multivariate_Gaussian_probability_variables_lidar.png"/>

- in the case of lidar, the variables in question would be the $x$ and $y$ position of the landmark and vehicle coordinates
- the diagonal terms of the covariance matrix are the variance of each variable, which is the standard deviation of the variable squared

<img src="resources/lidar_oval_covariance matrices.png"/>

- here, the orange ovals around each lidar measurement are geometric representations of some covariance matrices
- notice how the covariance changes as you change the diagonal terms
  - as the $x$ variance term increases, the uncertainty in the $x$ direction also increases
  - note that feature measurements are usually given in vehicle coordinates, where the $x$-axis points in the direction of the car's heading and the $y$-axis points to the left of the car
  - similarly, as the $y$ variance term increases, the uncertainty in the $y$ direction also increases
  - you can think of the covariance matrix as an inverse matrix of weights
- the smaller the diagonal term for a certain variable, the more you can trust this variable in the measurement and the higher the weight we can put on it


- the off diagonal terms of the covariance matrix represent the correlation between the two variables
  - for instance, if $x$ increases as $y$ increases, you could represent that in the off diagonal terms of this matrix
- for the project, we will assume the different variables in the sensor measurement are independent, and therefore the off diagonal terms are $0$
  - however, this is often not the case in practice

<img src="resources/lidar_oval_covariance matrices_off_diagonal.png"/>

- here is a geometric representation of some covariance matrices with off diagonal terms
- notice how the ovals look different from the covariances before that only had non-zero diagonal terms
- they now show a positive or negative correlation between $x$ and $y$


- after you update the weights for each particle, you will have to resample the particles with probability proportional to these weights

# Calculating Error

- once you finish implementing your particle filter, I will ask you to assess how accurate your position estimates were
- for your project, I have given you the ground truth position of the car for every time step


- we will now discuss two different ways to quantify the difference between your results and the ground truth
  - one way you could report your error is to take the weighted average error of all the particles
    - to do this, simply take the root squared error between each particle and ground truth, and multiply it by the particle's weight, and then, you'll have to divide by the sum of the weights
    - $error_{weighted} = \dfrac{\sum_{i=1}^{M}w_i \sqrt{|p_i-g|}}{\sum_{i=1}^{M}w_i}$
      - $w_i$ is weight of particle $i$
      - $p_i$ is particle $i$ (state vector)
      - $g$ is ground truth (state vector)
    - remember that in this equation, $p_i$ and $g$ are vectors that contain the $x$ position, $y$ position, and yaw of the car in map coordinates
  - another possibility is to just look at the best, or the highest-weighted particle, and simply take the root squared error of this particle

## Instructor Notes

- note that for a vector $v=(x,y)$, $∣v∣$ is used here to denote the vector length or magnitude = $\sqrt{x^2 + y^2}$
- for a difference between two vectors, $v = {(x_v,y_v)}$ and $w = {(x_w,y_w)}$, the magnitude $∣v−w∣$ is also the Euclidean distance between these two vectors = ($\sqrt{(x_v - x_w)^2 + (y_v - y_w)^2}$
- in this case $\sqrt{∣p_i - g∣}$ is meant to note the root squared error of a particular particle, resulting in the general form: $\sqrt{(x_p - x_g)^2 + (y_p - y_g)^2}$ where:
  - position RMSE = $\sqrt{(x_p - x_g)^2 + (y_p - y_g)^2}$
  - theta RMSE = $\sqrt{(\theta_p - \theta_g)^2}$

- **Q:** Given that the car’s ground truth position was $(x, y, \theta) = (5.2 m, 19.3 m, \pi/16)$ and the best particle’s position was $(x, y, \theta) = (5 m, 18.7 m, \pi/8)$, what is the error?
- **A:** $RMSE(x,y) = 0.63 $, $RMSE(\theta) = \dfrac{\pi}{16}$

# Transformations and Associations

- in the project you will need to correctly perform observation measurement transformations, along with identifying measurement landmark associations in order to correctly calculate each particle's weight
- remember, our ultimate goal is to find a weight parameter for each particle that represents how well that particle fits to being in the same location as the actual car


- in the quizzes that follow we will be given a single particle with its position and heading along with the car's observation measurements
- we will first need to transform the car's measurements from its local car coordinate system to the map's coordinate system
- next, each measurement will need to be associated with a landmark identifier, for this part we will take the closest landmark to each transformed observation
- finally, we will use this information to calculate the weight value of the particle

<img src="resources/transformations_and_associations.png"/>

- in the graph above we have a car (ground truth position) that observes three nearby landmarks, each one labeled $OBS1, OBS2, OBS3$
- each observation measurement has $x$, and $y$ values in the car's coordinate system
- we have a particle $P$ (estimated position of the car) above with position $(4,5)$ on the map with heading $-90$ degrees
- the first task is to transform each observation marker from the vehicle's coordinates to the map's coordinates, with respect to our particle

# Converting Landmark Observations

- in the next few concepts, we will walk through critical steps for localizing our car by implementing a particle filter
- our ultimate goal is determine how well each of our particle positions represents the actual position of our car
- to do this, we must use car sensor and map inputs to weigh each particle
- in the next few concepts, we will walk through this process for a single particle by:
  - transforming car sensor landmark observations from the car coordinate system to the map coordinate system
  - associating these transformed observations with the nearest landmark on the map
  - updating our particle weight by applying the multivariate Gaussian probability density function for each measurement, then combining the probabilities of all the measurements by taking their product
    - this final weight is our posterior probability

- graph above depicts a car observing three nearby landmarks, L5, L1, and L2
- the car sees them as observation one, observation two, and observation three
- remember that by convention, we define the car's coordinates such as $x$ is pointed in the direction of the vehicle's heading with $y$ rotated plus $90$ degrees or to the left perpendicular as depicted here


- the blue box in this figure represents a particle whose weight we want to update
- to do this, we will transform the car observations to map coordinates
- we can achieve this transformation by passing our data through a trigonometric function that maps car coordinates, to map coordinates
  - this function is called a **homogeneous transformation**, and is composed of a rotation in a translation
  - essentially, we'll need to rotate the maps frame to match the particles point of view
  - then we will move the origin of the frame to the location of the particle
  - we can do both of these steps in one matrix multiplication, using the homogeneous transformation


- we know that our particle has a heading of negative $90$ degrees and a map location of $(4, 5)$
- we can visualize this process by drawing a coordinate frame at the map origin, then determining the location of the particle frame in the map coordinate system
- to get to the particles frame from the map point of view, we first have to rotate by negative $90$ degrees, then we'll move it to the particles position


- by passing our observation coordinates $x_c$ and $y_c$, our particle coordinates $x_p$ and $y_p$ and a rotation angle theta of negative $90$ degrees, the homogeneous transformation right here does exactly what we just did
- this formulation with the extra one as a third row is known as homogeneous coordinates

<img src="resources/homogeneous_transformation.png"/>

- let's quickly break down this homogeneous transformation here
- the two by two matrix in the top left represents the rotation component about the $Z$ axis
- remember that we're only doing this in 2D right now
- the third column represents the translation component
- we complete the transformation by multiplying through by the observation in car coordinates
- to summarize, this homogeneous transformation is a transformation from car coordinates to map coordinates in the maps frame


- using our observations in the car coordinate system and our particle pose in the map system, we have everything we need to transform observations into map coordinates, and ultimately determine the final weight of our particle

- referring to the figures below:
  - suppose the map coordinate system (grey lines) and the vehicle coordinate system (orange lines) are offset, as depicted below
  - if we know the location of the observation in vehicle coordinates (orange lines), we would need to rotate the entire system, observation included, $-45$ degrees to find it in map coordinates (grey lines)
  - once this rotation is done, we can easily see the location of the observation in map coordinates
  
  
- Particle (blue dot) in Map Frame (grey)

<img src="resources/particle_in_map_frame.png"/>

- Particle (blue dot) in Vehicle Frame (orange)

<img src="resources/particle_in_vehicle_frame.png"/>

# Landmarks

- in the graph above we have a car (ground truth position) that observes three nearby landmarks, each one labeled $OBS1, OBS2, OBS3$
- each observation measurement has $x$, and $y$ values in the car's coordinate system
- we have a particle $P$ (estimated position of the car) above with position $(4,5)$ on the map with heading $-90$ degrees
- the first task is to transform each observation marker from the vehicle's coordinates to the map's coordinates, with respect to our particle


- there is a generalized trigonometric function that given any particle position and heading along with any observation measurement $(x_{obs},y_{obs})$ will output the transformed observation $(x_{map},y_{map})$ for that particle
- in other words, we need to map particle coordinates to map coordinates, by passing particle and observation coordinates through a function
- $Transformed Observation (x_{map}, y_{map}) = func(x_{particle}, y_{particle}, heading\_particle, x_{obs}, y_{obs})$
- you will need to derive and use this function to efficiently calculate particle observation transformations in the project


- some excellent resources discussing coordinate transformation and rotation matrices:
  - [Coordinate Transformations](http://farside.ph.utexas.edu/teaching/336k/Newtonhtml/node153.html)
  - [Coordinate Transformation Under Rotation](https://www.miniphysics.com/coordinate-transformation-under-rotation.html)

## Resources, Hints, and Tips

- without implementation of localization methods the car does not know where it is within an acceptable level of precision
- the car knows particle coordinates and observation coordinates
- the objective is to use the particle coordinates and heading to transform the car's frame of reference to the map's frame of reference, associate the observations, then use the associated observations in the map domain to update the particle weight


- since we know the coordinates of the particle from the car's frame of reference we can use this information and a matrix rotation/translation to transform each observation from the car frame of reference to the map frame of reference
- the particle is at $(4,5)$ in the map coordinate system with a heading of $-90$ degrees
- the figure indicates the heading by depicting the particle $x$-axis as pointing down (blue arrow)
- this is critical to understanding the matrix transformation we are about to perform


- by convention we define the car coordinate system with $x$ pointing up and $y$ rotated from $x$ by $\dfrac{\pi}{2}$ ($+90$ degrees)
- this is another way of saying that $y$ is perpendicular and to the left of $x$
- when $x$ is pointing down, we have the mirror of this, with $y$ perpendicular to the right
- to visualize this make an L with your left index finger and thumb with palm facing away from you, this is our map frame of reference
- point the thumb towards the ceiling, this is the car coordinate convention, now point your thumb down, this is the orientation of the particle at $(4,5)$


- now consider the map frame of reference (make an L with your left index finger and thumb as above), this is a typical presentation of Cartesian coordinates, with $x$ pointing right and $y$ perpendicular to the left, pointing up
- if we rotate our thumb down we have the particle orientation
- to get back to the map orientation we must rotate counterclockwise by $90$ degrees (thumb from pointing down back to pointing right)
- try this a few times with your left hand--notice that particle to map is a counterclockwise rotation ($+90$ degrees) and map to particle is a clockwise rotation ($-90$ degrees)


- the most straight forward way of rotating and translating coordinates is through homogenous transformation matrix (see below) using the angle of rotation required to get to the particle’s frame from the map’s point of view, $-90$ degrees
- this way we can use theta directly
- the alternative, which we will not cover here is to is to use theta and a transformation matrix from the particle frame to the map frame


- [this video](https://youtu.be/h11ljFJeaLo) is a great resource for developing a deeper understanding of how to solve this transformation problem - it covers the rotation transformation, and from there you just need to perform a translation

- observations in the car coordinate system can be transformed into map coordinates $(x_m, y_m)$ by passing car observation coordinates $(x_c, y_c)$, map particle coordinates $(x_p, y_p)$, and our rotation angle ($-90$ degrees) through a homogenous transformation matrix
  - this homogenous transformation matrix, shown below, performs rotation and translation
  - $\left[ \begin{array}{c} x_m \\ y_m \\ 1 \end{array} \right] = \begin{bmatrix} \cos\theta & -\sin\theta & \text{x}_p \\ \sin\theta & \cos\theta & \text{y}_p \\ 0 & 0 & 1 \end{bmatrix} \times \left[ \begin{array}{c} x_c \\ y_c \\ 1 \end{array} \right]$
  - [matrix multiplication](https://www.mathsisfun.com/algebra/matrix-multiplying.html) results in:
    - $x_m= x_p + (\cos\theta \times x_c) - (\sin\theta \times y _c)$
    - $y_m= y_p + (\sin\theta \times x_c) + (\cos\theta \times y_c)$

- **Q:** OBS1 is the sensor observation reported from the sensor. As noted in the figure, OBS1 is (2,2). What is the position of OBS1 in map coordinates (x_map, y_map)? Enter your answer in parenthesis with the x value separated from the y value with a comma and no spaces.
- **A:** $(6,3)$. $(x_p, y_p) = (4, 5)$


- **Q:** OBS2 is the sensor observation reported from the sensor. As noted in the figure, OBS2 is (3,-2). What is the position of OBS2 in map coordinates (x_map, y_map)? Enter your answer in parenthesis with the x value separated from the y value with a comma and no spaces. The answer does not match L2 - if you think it does, you need to check your work!
- **A:** $(2,2)$.


- **Q:** OBS3 is the sensor observation reported from the sensor. As noted in the figure, OBS3 is (0,-4). What is the position of OBS3 in map coordinates (x_map, y_map)? Enter your answer in parenthesis with the x value separated from the y value with a comma and no spaces.
- **A:** $(0,5)$.


- code is available in `code/02_landmarks_quiz/`

# Association

- now that observations have been transformed into the map's coordinate space, the next step is to associate each transformed observation with a landmark identifier
- in the map exercise above we have 5 total landmarks each identified as L1, L2, L3, L4, L5, and each with a known map location
- we need to associate each transformed observation TOBS1, TOBS2, TOBS3 with one of these 5 identifiers
- to do this we must associate the closest landmark to each transformed observation.


- as a reminder: `TOBS1` = $(6,3)$, `TOBS2` = $(2,2)$ and `TOBS3` = $(0,5)$

- **Q:** Which landmark identifier is transformed OBS1 associated with? Answer either L1, L2, L3, L4, or L5.
- **A:** `TOBS1` is closest to L1.


- **Q:** Which landmark identifier is transformed OBS2 associated with? Answer either L1, L2, L3, L4, or L5.
- **A:** `TOBS2` is closest to L2.


- **Q:** Which landmark identifier is transformed OBS3 associated with? Answer either L1, L2, L3, L4, or L5.
- **A:** `TOBS3` can be associated with either L2, or L5, since they are the same distance apart.

# Particle Weights

- now we that we have done the measurement transformations and associations, we have all the pieces we need to calculate the particle's final weight
- the particles final weight will be calculated as the product of each measurement's Multivariate-Gaussian probability density


- the Multivariate-Gaussian probability density has two dimensions, $x$ and $y$
- the mean of the Multivariate-Gaussian is the measurement's associated landmark position and the Multivariate-Gaussian's standard deviation is described by our initial uncertainty in the $x$ and $y$ ranges
- the Multivariate-Gaussian is evaluated at the point of the transformed measurement's position
- the formula for the Multivariate-Gaussian is: $P(x,y) =\frac{1}{2\pi \sigma_{x}\sigma_{y}}e^{-(\frac{(x-\mu_x)^2}{2\sigma_x^2}+\frac{(y-\mu_y)^2}{2\sigma_y^2})}$

- to complete the next set of quizzes, calculate each measurement's Multivariate-Gaussian probability density using the formula above and the previously calculated values
- in this example the standard deviation for both $x$ and $y$ is $0.3$
- to get the final weight just multiply all the calculated measurement probabilities together


- note that $x$ and $y$ are the observations in map coordinates from the landmarks quiz and $\mu_x$, $\mu_y$ are the coordinates of the nearest landmarks
- these should correspond to the correct responses from previous quizzes


- code is available in `code/03_particle_weights_quiz/`

- **Q:** What is the calculated Multivariate-Gaussian probability density of OBS1? Write your answer in scientific notation with an accuracy of two decimal places, for example 3.14E-15.
- **A:** OBS1 weight should be around 0.00683644777551 rounding to 6.84E-3.


- **Q:** What is the calculated Multivariate-Gaussian probability density of OBS2? Write your answer in scientific notation with an accuracy of two decimal places, for example 3.14E-15.
- **A:** OBS2 weight should be around 0.00683644777551, rounding to 6.84E-3.


- **Q:** What is the calculated Multivariate-Gaussian probability density of OBS3? Write your answer in scientific notation with an accuracy of two decimal places, for example 3.14E-15.
- **A:** OBS3 weight should be around 9.83184874151e-49, rounding to 9.83E-49.


- **Q:** What is the particle's final weight? Write your answer in scientific notation with an accuracy of two decimal places, for example 3.14E-15.
- **A:** The final weight should be around 4.59112934464959e-53, rounding to 4.60E-53 (the above three multiplied together).

# Additional Resources on Localization

- nice work reaching the end of the localization content!
- while you still have the project left to do here, we're also providing some additional resources and recent research on the topic that you can come back to if you have time later on


- all of these are completely optional reading - you could spend hours reading through the entirety of these!
- we suggest moving onto the project first so you have what you’ve learned fresh on your mind, before coming back to check these out


- we've categorized these papers to hopefully help you narrow down which ones might be of interest, as well as highlighted a couple key reads by category by including their Abstract section, which summarizes the paper

## Simultaneous Localization and Mapping (SLAM)

- the below papers cover Simultaneous Localization and Mapping (SLAM) - which as the name suggests, combines localization and mapping into a single algorithm without a map created beforehand

Past, Present, and Future of Simultaneous Localization And Mapping: Towards the Robust-Perception Age by C. Cadena, et. al. https://arxiv.org/abs/1606.05830

***Abstract:*** *Simultaneous Localization and Mapping (SLAM) consists in the concurrent construction of a model of the environment (the map), and the estimation of the state of the robot moving within it. The SLAM community has made astonishing progress over the last 30 years, enabling large-scale real-world applications, and witnessing a steady transition of this technology to industry. We survey the current state of SLAM. We start by presenting what is now the de-facto standard formulation for SLAM. We then review related work, covering a broad set of topics including robustness and scalability in long-term mapping, metric and semantic representations for mapping, theoretical performance guarantees, active SLAM and exploration, and other new frontiers. [...]*

Navigating the Landscape for Real-time Localisation and Mapping for Robotics and Virtual and Augmented Reality by S. Saeedi, et. al. https://arxiv.org/abs/1808.06352

***Abstract:*** *Visual understanding of 3D environments in real-time, at low power, is a huge computational challenge. Often referred to as SLAM (Simultaneous Localisation and Mapping), it is central to applications spanning domestic and industrial robotics, autonomous vehicles, virtual and augmented reality. This paper describes the results of a major research effort to assemble the algorithms, architectures, tools, and systems software needed to enable delivery of SLAM, by supporting applications specialists in selecting and configuring the appropriate algorithm and the appropriate hardware, and compilation pathway, to meet their performance, accuracy, and energy consumption goals. [...]*

## Other Methods

- the below paper from Udacity's founder Sebastian Thrun, while from 2002, is still relevant for many different methods of mapping used today in robotics

Robotic Mapping: A Survey by S. Thrun http://robots.stanford.edu/papers/thrun.mapping-tr.pdf

***Abstract:*** *This article provides a comprehensive introduction into the field of robotic mapping, with a focus on indoor mapping. It describes and compares various probabilistic techniques, as they are presently being applied to a vast array of mobile robot mapping problems. The history of robotic mapping is also described, along with an extensive list of open research problems.*